In [288]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

In [332]:
baseUrl = 'https://www.nfl.com'

In [314]:
playerPayload = {"stats":"/stats","subject":"player-stats","category":"category",
                 "statisticCategory":"rushing","season":"2019",
                 "seasonType":"REG",
                 "list":"all",
                 "specificCategory":"rushingYards",
                 "order":"DESC"}

teamPayload = {"stats":"/stats","subject":"team-stats","category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}

In [331]:
payload = teamPayload

url = 'https://www.nfl.com'
for key in payload:
    if payload[key]:
        url += payload[key]
        url += '/'
response = requests.get(url)
print(response.status_code,response.url)
soup = BeautifulSoup(response.text,'html.parser')

200 https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


In [316]:
columnHeader = []
header = soup.find('table').find_all('th')
for col in header:
    columnHeader.append(col.text)
columnHeader = [c.strip('\n') for c in columnHeader]
print(columnHeader)

['Team', 'Att', 'Cmp', 'Cmp %', 'Yds/Att', 'Pass Yds', 'TD', 'INT', 'Rate', '1st', '1st%', '20+', '40+', 'Lng', 'Sck', 'SckY']


In [324]:
tableRows = []
pagingText = True

while pagingText:
    time.sleep(2)
    response = requests.get(url)
    print("url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            cols = [ele.rstrip('\n') for ele in cols]

            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
    if pagingText:
        url = baseUrl + pagingText.get('href')

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


In [330]:
resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
# remove \n whitespace from team names
find = re.compile(r"^(\n*).*")
resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
pd.DataFrame.head(resultsDf)

,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Redskins,479,298,62.2,6.7,3205,18,13,83,154,32.2,38,6,75T,50,393
1,Buccaneers,630,382,60.6,8.1,5127,33,30,84.1,244,38.7,75,13,71T,47,282
2,Seahawks,517,341,66,7.9,4110,31,6,105.3,190,36.8,55,10,60T,48,319
3,49ers,478,331,69.2,8.4,4029,28,13,103.1,195,40.8,62,8,75T,36,237
4,Chargers,597,394,66,7.8,4648,24,20,89,220,36.9,57,12,84T,34,222


In [334]:
def getTable(url, payload): 
    tableRows = []
    pagingText = True
    
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    
    # get column names
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = []
    header = soup.find('table').find_all('th')
    for col in header:
        columnHeader.append(col.text)
    columnHeader = [c.strip('\n') for c in columnHeader]

    # search through pages and append to tableRows
    while pagingText:
        time.sleep(2)
        response = requests.get(url)
        print("url:",response.url)
        soup = BeautifulSoup(response.text,'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')

        for row in rows:
            cols = row.find_all('td')
            if cols:
                cols = [ele.text.strip() for ele in cols]
                cols = [ele.rstrip('\n') for ele in cols]
                tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
        pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
        if pagingText:
            url = baseUrl + pagingText.get('href')
    
    resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
    find = re.compile(r"^(\n*).*")
    resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
    return resultsDf

In [337]:
teamPayload = {"stats":"/stats","subject":"team-stats","category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}
df = getTable(baseUrl,teamPayload)
pd.DataFrame.head(df)

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Redskins,479,298,62.2,6.7,3205,18,13,83,154,32.2,38,6,75T,50,393
1,Buccaneers,630,382,60.6,8.1,5127,33,30,84.1,244,38.7,75,13,71T,47,282
2,Seahawks,517,341,66,7.9,4110,31,6,105.3,190,36.8,55,10,60T,48,319
3,49ers,478,331,69.2,8.4,4029,28,13,103.1,195,40.8,62,8,75T,36,237
4,Chargers,597,394,66,7.8,4648,24,20,89,220,36.9,57,12,84T,34,222
